### git repository 불러오기

In [ ]:
!git clone https://github.com/ultralytics/yolov3.git

### Library 불러오기

In [ ]:
import sys
import os

sys.path.append("./yolov3")

In [ ]:
from pycocotools.coco import COCO
import numpy as np
import cv2

import albumentations as A
from albumentations.pytorch import ToTensorV2
from utils.loss import ComputeLoss

import yaml
import torch
from models.yolo import Model
from pycocotools.cocoeval import COCOeval
import pandas as pd

from torch.utils.data import DataLoader, Dataset
import torchvision
from utils.general import (
    coco80_to_coco91_class,
    check_dataset,
    check_file,
    check_img_size,
    check_requirements,
    box_iou,
    non_max_suppression,
    scale_coords,
    xyxy2xywh,
    xywh2xyxy,
    set_logging,
    increment_path,
    colorstr,
)
from tqdm import tqdm

### Custom 데이터셋 정의

In [ ]:
class CustomDataset(Dataset):
    """
    data_dir: data가 존재하는 폴더 경로
    transforms: data transform (resize, crop, Totensor, etc,,,)
    """

    def __init__(self, annotation, data_dir, transforms=None, image_size=512):
        super().__init__()
        self.data_dir = data_dir
        self.coco = COCO(annotation)
        self.image_size = image_size
        self.predictions = {
            "images": self.coco.dataset["images"].copy(),
            "categories": self.coco.dataset["categories"].copy(),
            "annotations": None,
        }
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.coco.getImgIds(imgIds=index)
        image_info = self.coco.loadImgs(image_id)[0]
        image = cv2.imread(os.path.join(self.data_dir, image_info["file_name"]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        ann_ids = self.coco.getAnnIds(imgIds=image_info["id"])
        anns = self.coco.loadAnns(ann_ids)

        # boxes (x, y, w, h)
        boxes = np.array([x["bbox"] for x in anns])

        # boxes (x_min, y_min, x_max, y_max)
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        boxes[:, 0] /= int(self.image_size)
        boxes[:, 1] /= int(self.image_size)
        boxes[:, 2] /= int(self.image_size)
        boxes[:, 3] /= int(self.image_size)

        labels = list([x["category_id"] for x in anns])
        areas = np.array([x["area"] for x in anns])
        areas = torch.as_tensor(areas, dtype=torch.float32)

        is_crowds = np.array([x["iscrowd"] for x in anns])
        is_crowds = torch.as_tensor(is_crowds, dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": torch.tensor([index]),
            "area": areas,
            "iscrowd": is_crowds,
        }

        # transform
        if self.transforms:
            sample = {"image": image, "bboxes": target["boxes"], "labels": labels}
            sample = self.transforms(**sample)
            image = sample["image"]
            target["boxes"] = [list(box) for box in sample["bboxes"]]

        result = []
        labels_out = torch.zeros((len(labels), 6))
        for i in range(len(labels)):
            result.append(
                [
                    labels[i],
                    target["boxes"][i][0],
                    target["boxes"][i][1],
                    target["boxes"][i][2],
                    target["boxes"][i][3],
                ]
            )
        result = np.array(result)

        if len(labels):
            labels_out[:, 1:] = torch.from_numpy(result)

        return image, labels_out, image_id

    def __len__(self) -> int:
        return len(self.coco.getImgIds())

In [ ]:
def get_train_transform():
    return A.Compose(
        [A.Resize(512, 512), A.Flip(p=0.5), ToTensorV2(p=1.0)],
        bbox_params={"format": "pascal_voc", "label_fields": ["labels"]},
    )


def get_valid_transform():
    return A.Compose(
        [ToTensorV2(p=1.0)],
        bbox_params={"format": "pascal_voc", "label_fields": ["labels"]},
    )

### Utils

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0


def collate_fn(batch):
    img, label, image_id = zip(*batch)  # transposed
    for i, l in enumerate(label):
        l[:, 0] = i  # add target image index for build_targets()
    return torch.stack(img, 0), torch.cat(label, 0), image_id

### Training

In [ ]:
def train_fn(num_epochs, train_data_loader, optimizer, model, device):
    model.gr = 1.0
    itr = 1
    loss_hist = Averager()
    compute_loss = ComputeLoss(model)
    best_loss = 1000
    for epoch in range(num_epochs):
        loss_hist.reset()

        for images, targets, image_ids in tqdm(train_data_loader):
            images = torch.as_tensor([image.numpy() for image in images]).to(device)
            pred = model(images)

            losses, loss_items = compute_loss(
                pred, targets.to(device)
            )  # loss scaled by batch_size

            loss_value = losses.item()

            loss_hist.send(loss_value)

            # backward
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

        print(f"Epoch #{epoch} loss: {loss_hist.value}")
        if loss_hist.value < best_loss:
            torch.save(model.state_dict(), f"./yolov3/weights/yolov3.pth")
            best_loss = loss_hist.value

### Main

In [ ]:
def main():
    data_dir = "../../dataset"
    annotation = "../../dataset/train.json"
    train_dataset = CustomDataset(annotation, data_dir, get_train_transform())

    train_data_loader = DataLoader(
        train_dataset, batch_size=8, shuffle=False, num_workers=4, collate_fn=collate_fn
    )
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    print(device)
    # load a model;

    num_classes = 10
    model = Model(cfg="./yolov3/models/yolov3.yaml", ch=3, nc=num_classes)  # create

    # get number of input features for the classifier
    model.to(device)

    # Hyperparameters
    with open("./yolov3/data/hyps/hyp.scratch.yaml") as f:
        hyp = yaml.load(f, Loader=yaml.SafeLoader)  # load hyps
    model.hyp = hyp
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

    num_epochs = 10

    train_fn(num_epochs, train_data_loader, optimizer, model, device)

In [ ]:
if __name__ == "__main__":
    main()

###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.
